In this notebook we will be analyzing plate reader data for the dose response of 1x and 2x naive T7 RNAP expression strains on T7 RNAP driven GFP procution and its associated burden as shown in Supplementary Figure 12.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize
import murraylab_tools.biotek as btek
import itertools

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from cycler import cycler
import numpy as np


sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 1.5})
sns.set_style("ticks")
sns.set_style({"xtick.direction": "in","ytick.direction": "in"})


mpl.rc('axes', prop_cycle=(cycler('color', ['r', 'k', 'b','g','y','m','c']) ))

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42


tw = 1.5
sns.set_style({"xtick.major.size": 3, "ytick.major.size": 3,
               "xtick.minor.size": 2, "ytick.minor.size": 2,
               'axes.labelsize': 16, 'axes.titlesize': 16,
               'xtick.major.width': tw, 'xtick.minor.width': tw,
               'ytick.major.width': tw, 'ytick.minor.width': tw})

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)
mpl.rc('axes', linewidth=1.5)
mpl.rc('legend', fontsize=14)
mpl.rc('legend', frameon=False)
mpl.rc('figure', figsize=(8.5,15))
%matplotlib qt

In [2]:
# generate csv with tidy df
btek.tidy_biotek_data('./20210821_naive_HCK_IPTG_gradient.csv',convert_to_uM=False,volume=300, supplementary_filename='20210821_metadata.csv')

/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/murraylab_tools/biotek/biotek.py:305: UserWarning: Unknown plate reader id '2011231C'; will not attempt to calculate molarity concentrations.
  "concentrations.") % line[1])
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/murraylab_tools/biotek/biotek.py:473: UserWarning: No supplementary data for well G1; throwing out data for that well.
  % well_name)
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/murraylab_tools/biotek/biotek.py:473: UserWarning: No supplementary data for well G2; throwing out data for that well.
  % well_name)
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/murraylab_tools/biotek/biotek.py:473: UserWarning: No supplementary data for well G3; throwing out data for that well.
  % well_name)
/Users/rorywilliams/opt/anaconda3/lib/python3.7/site-packages/murraylab_tools/biotek/biotek.py:473: UserWarning: No supplementary data for well G4; throwing out data for that w

In [3]:
# load tidy data frame and generate OD normalized fluorsecence data
naive_data = pd.read_csv('20210821_naive_HCK_IPTG_gradient_tidy.csv')
naive_data = naive_data.loc[naive_data['Time (hr)']<=8,:]
naive_data_norm = btek.normalize(naive_data)

In [4]:
# get endpoint and average/std data
naive_OD = naive_data.loc[naive_data.Channel=='OD600',:]
naive_GFP = naive_data.loc[(naive_data.Channel=='sfGFP')&\
                           (naive_data.Gain==50),:]
naive_GFP_norm = naive_data_norm.loc[(naive_data_norm.Channel=='sfGFP')&\
                                     (naive_data_norm.Gain==50),:]
naive_GFP_norm_ep = naive_GFP_norm.loc[naive_GFP_norm['Time (hr)']==naive_GFP_norm['Time (hr)'].max(),:]
naive_GFP_norm_ep = naive_GFP_norm_ep.loc[(naive_GFP_norm_ep.iptg!=40)&\
                                                (naive_GFP_norm_ep.iptg<60),:]
naive_GFP_norm_ep_avg = naive_GFP_norm_ep.groupby(['strain','iptg'],as_index=False)['Measurement'].mean()
naive_GFP_norm_ep_avg['std'] = naive_GFP_norm_ep.groupby(['strain','iptg'])['Measurement'].std().values
naive_OD_avg = naive_OD.groupby(['strain','iptg','Time (hr)'], as_index=False).agg(
                      {'Measurement':['mean','std']})
naive_GFP_avg = naive_GFP.groupby(['strain','iptg','Time (hr)'], as_index=False).agg(
                      {'Measurement':['mean','std']})

In [7]:
# determine fold change of normalized fluorescence relative to the uninduced 1x naive case
# this is not plotted, but is referenced in the manuscript
naive_GFP_norm_ep_avg['norm_Measurement'] = naive_GFP_norm_ep_avg['Measurement']/naive_GFP_norm_ep_avg['Measurement'].values[0]
naive_GFP_norm_ep_avg

,strain,iptg,Measurement,std,norm_Measurement
0,naive 1x,0,3832.709931,60.465129,1.000000
1,naive 1x,5,5294.990806,8.711872,1.381527
2,naive 1x,10,7708.323101,198.891839,2.011194
3,naive 1x,20,14544.662118,190.450730,3.794877
4,naive 1x,30,18765.034802,366.635059,4.896023
5,naive 1x,50,24728.192582,672.637718,6.451882
6,naive 2x,0,3499.422760,358.905475,0.913041
7,naive 2x,5,6791.488944,2510.168705,1.771981
8,naive 2x,10,11994.701700,5218.850448,3.129562
9,naive 2x,20,18578.249236,1164.749969,4.847288


Below we plot GFP productiona and cell denstity data (average with SD shaded) as shown in Supplementary Figure 12A.

In [9]:
fig, ax = plt.subplots(2,2,figsize=(9,9),sharex=True,sharey='row')
iptgs = [0,10,20,30,50]
sns.set_palette('colorblind',10)

for i, strain in enumerate(['naive 1x','naive 2x']):
    ax[0,i].set_title(strain,fontsize=14)
    for j, iptg in enumerate(iptgs):
        t = naive_GFP_avg.loc[(naive_GFP_avg.strain==strain)&\
                         (naive_GFP_avg.iptg==iptg),'Time (hr)']
        y_mean = naive_GFP_avg.loc[(naive_GFP_avg.strain==strain)&\
                         (naive_GFP_avg.iptg==iptg),'Measurement']['mean']
        y_std = naive_GFP_avg.loc[(naive_GFP_avg.strain==strain)&\
                         (naive_GFP_avg.iptg==iptg),'Measurement']['std']
        ax[0,i].plot(t,y_mean,color=sns.color_palette()[j])
        t = naive_OD_avg.loc[(naive_OD_avg.strain==strain)&\
                         (naive_OD_avg.iptg==iptg),'Time (hr)']
        y_mean = naive_OD_avg.loc[(naive_OD_avg.strain==strain)&\
                         (naive_OD_avg.iptg==iptg),'Measurement']['mean']
        y_std = naive_OD_avg.loc[(naive_OD_avg.strain==strain)&\
                         (naive_OD_avg.iptg==iptg),'Measurement']['std']
        ax[1,i].plot(t,y_mean,color=sns.color_palette()[j])
for i, strain in enumerate(['naive 1x','naive 2x']):
    for j, iptg in enumerate(iptgs):
        t = naive_GFP_avg.loc[(naive_GFP_avg.strain==strain)&\
                         (naive_GFP_avg.iptg==iptg),'Time (hr)']
        y_mean = naive_GFP_avg.loc[(naive_GFP_avg.strain==strain)&\
                         (naive_GFP_avg.iptg==iptg),'Measurement']['mean']
        y_std = naive_GFP_avg.loc[(naive_GFP_avg.strain==strain)&\
                         (naive_GFP_avg.iptg==iptg),'Measurement']['std']
        ax[0,i].fill_between(t,y_mean-y_std,y_mean+y_std,color=sns.color_palette()[j],alpha=0.2)
        t = naive_OD_avg.loc[(naive_OD_avg.strain==strain)&\
                         (naive_OD_avg.iptg==iptg),'Time (hr)']
        y_mean = naive_OD_avg.loc[(naive_OD_avg.strain==strain)&\
                         (naive_OD_avg.iptg==iptg),'Measurement']['mean']
        y_std = naive_OD_avg.loc[(naive_OD_avg.strain==strain)&\
                         (naive_OD_avg.iptg==iptg),'Measurement']['std']
        ax[1,i].fill_between(t,y_mean-y_std,y_mean+y_std,color=sns.color_palette()[j],alpha=0.2)
ax[0,0].legend(iptgs,loc='upper left',fontsize=12,title='IPTG',title_fontsize=12)
ax[0,0].set_ylabel('GFP fluorescence (a.u.)',fontsize=14)
ax[1,0].set_ylabel('OD600',fontsize=14)
ax[1,0].set_xlabel('time (h)',fontsize=14)
ax[1,1].set_xlabel('time (h)',fontsize=14)
plt.savefig('20220809_naive1x_2x_gfp_od_sdshade.pdf')
        

Below we adopt code from the murray_lab_tools.biotek package (courtesy of Sam Clamons) for fitting growth rates on all wells. We make the modification of using an exponential growth function, and trim data strictly by OD value before fitting.

In [11]:
def summarize_growth(channel_df, fixed_init = None, growth_threshold = None,
                     verbose = False, growth='logistic'):
    '''
    Summarizes the growth characteristics of OD curves from a dataframe of
    Biotek data. Performs the following summaries on each well:
        * Fits OD curve to a logistic-plus-floor or exponential-plus-floor, finding an initial value, a
            noise floor, a rate constant (R, not to be interpreted directly),
            and a population maximum. The rate parameter has time units of
            hours.
        * Optionally finds the time when the population crosses some fraction of
            maximum population. By default, does not calculate this -- set
            the growth_threshold parameter to add this calculation.
    Params:
        df -- A DataFrame of Biotek data with at least one channel of growth
                data.
        channel -- The name of the channel with growth data. Should be a channel
                    with only one Gain, or this will do weird things.
        growth_threshold -- If set, determines the fraction of of total
                                population to find the time of, i.e., if
                                growth_threshold = 0.25, this function will
                                report the time that each well crossed 25%
                                of the total population size for that well.
        fixed_init -- Sets a fixed value for the initial population parameter.
                        If None, this value is optimized with the rest of the
                        parameters.
        verbose -- Iff True, prints some hints about what it's doing. Use if it's taking
                    a while and you want to make sure it's making progress. Default False.
    Returns: A new dataframe where each row summarizes the growth
                characteristics of one from the original dataframe.
    '''

    # Split dataframe into a list of dataframes for individual wells.
    well_dfs = [channel_df[channel_df.Well == w] \
                for w in channel_df.Well.unique()]
    measurement_summary_rows = \
        list(map(lambda df: summarize_single_well_growth(df, growth_threshold,
                                                         fixed_init, verbose,growth),
                 well_dfs))
    return pd.DataFrame(measurement_summary_rows)

def summarize_single_well_growth(well_df, growth_threshold = None,
                                 fixed_init = None, verbose = False,growth='logistic'):
    '''
    Summarizes the growth characteristics of a single well's worth of dataframe,
    returning the results as a dictionary describing a single dataframe line.
    See summarize_growth for measurement details.
    This function is intended as a helper function for summarize_growth; it uses
    the helper function logistic_growth as a growth model.
    Params:
        well_df -- A DataFrame of Biotek data from a single well and a single
                channel (presumably an OD channel).
        growth_threshold -- If set, determines the fraction of of total
                                population to find the time of, i.e., if
                                growth_threshold = 0.25, this function will
                                report the time that each well crossed 25%
                                of the total population size for that well.
        fixed_init -- Sets a fixed value for the initial population parameter.
                        If None, this value is optimized with the rest of the
                        parameters.
        verbose -- Iff True, print some hints about how it's progressing.
    Returns: A dictionary containing the well name, growth characteristics, and
                any supplementary data from df.
    '''
    well_df.reset_index(inplace = True)
    if growth_threshold is not None:
        well_df = well_df.loc[well_df.Measurement<growth_threshold,:]
    if verbose:
        print("Summarizing from well %s" % well_df.Well[0])

    # Some empirically-reasonable guesses for most growth experiments.
    
    if growth == 'exponential':
        if fixed_init == None:
            param_guess = (1.3, 0.001, 0)
            opt_func = exp_growth
        else:
            param_guess = (1.3, 0.001)
            opt_func = lambda t, r, f: exp_growth(t, r, f, fixed_init)

        times = well_df["Time (hr)"]

        opt_params = scipy.optimize.curve_fit(opt_func, times,
                                              well_df["Measurement"],
                                              p0 = param_guess,
                                              maxfev = int(1e4))[0]

        opt_params = np.abs(opt_params)

        return_dict = dict()
        return_dict["Rate"]  = opt_params[0]
        return_dict["Floor"] = opt_params[1]
        if fixed_init == None:
            return_dict["Init"] = opt_params[2]
        else:
            return_dict["Init"] = fixed_init

        # Add supplemental data.
        for column in well_df.columns.values:
            if column in ["Channel", "Gain", "Time (sec)", "Time (hr)",
                          "Measurement", "Units", "Excitation", "Emission"]:
                continue
            return_dict[column] = well_df[column][0]

    return return_dict


def exp_growth(t, rate, floor, init):
    '''
    Model function for logistic growth with a noise floor.
    Params:
        t -- Time.
        rate -- Growth rate parameter.
        init -- Initial population.
        floor -- Noise floor (i.e., OD reading for zero cells).
    Returns: Model OD reading at the given time, for the given parameters.
    '''

    rate = np.abs(rate)
    init = np.abs(init)
    floor = np.abs(floor)
    return floor + init * np.exp(rate * t)

In [12]:
# we will only be fitting a subset of the iptg concentrations
naive_OD_calc = naive_OD.loc[(naive_OD.iptg!=40)&\
                             (naive_OD.iptg<60),:]
naive_growth_df = summarize_growth(naive_OD_calc,
                                  growth='exponential',growth_threshold=0.25)

In [13]:
naive_growth_avg = naive_growth_df.groupby(['strain','iptg'],as_index=False)['Rate','Init'].mean()
naive_growth_avg['std'] = naive_growth_df.groupby(['strain','iptg'])['Rate'].std().values

In [14]:
# determine burden relative to uninduced case for each strain
naive_growth_avg['burden'] = 0
for strain in naive_growth_avg.strain.unique():
    noiptg_growth = naive_growth_avg.loc[(naive_growth_avg.strain==strain)&
                                         (naive_growth_avg.iptg==0),'Rate'].values[0]
    naive_growth_avg.loc[naive_growth_avg.strain==strain,'burden'] = \
    (noiptg_growth - naive_growth_avg.loc[naive_growth_avg.strain==strain,'Rate'].values)/noiptg_growth

Below we plot growth rate as a function of IPTG concentration for navie 1x and 2x as shown in Supplementary Figure 12B. Average +/- SD and individual replicates plotted as small circles.

In [16]:
fix, ax = plt.subplots(figsize=(6,5))

for i, strain in enumerate(naive_growth_df.strain.unique()):
    ax.plot(naive_growth_df.loc[(naive_growth_df.strain==strain),'iptg'],
            naive_growth_df.loc[(naive_growth_df.strain==strain),'Rate'],
            '.',color=sns.color_palette()[i],ms=4)
    ax.errorbar(naive_growth_avg.loc[(naive_growth_avg.strain==strain),'iptg'],
                naive_growth_avg.loc[(naive_growth_avg.strain==strain),'Rate'],
               yerr=naive_growth_avg.loc[(naive_growth_avg.strain==strain),'std'],
               capsize=2,color=sns.color_palette()[i],linestyle='None',marker='s',ms=5)
    
# ax.set_ylim(0,2)
ax.legend(naive_growth_df.strain.unique(),loc='best')
ax.set_xlabel('IPTG ($\mu$M)',fontsize=14)
ax.set_ylabel('fitted growth rate (1/hr)',fontsize=12)
ax.legend(['naive 1x','naive 2x','naive 1x avg','naive 2x avg'],fontsize=12,loc='best')
plt.savefig('20210821_naive1x2x_HCK_iptg_growthrates_20220805.pdf')

Below we plot end point OD normalized GFP fluorescence as a function of IPTG concentration for navie 1x and 2x as shown in Supplementary Figure 12C. Average +/- SD and individual replicates plotted as small circles.

In [17]:
fix, ax = plt.subplots(figsize=(6,5))

for i, strain in enumerate(naive_GFP_norm_ep.strain.unique()):
    ax.plot(naive_GFP_norm_ep.loc[(naive_GFP_norm_ep.strain==strain),'iptg'],
            naive_GFP_norm_ep.loc[(naive_GFP_norm_ep.strain==strain),'Measurement'],
            '.',color=sns.color_palette()[i],ms=4)
    ax.errorbar(naive_GFP_norm_ep_avg.loc[(naive_GFP_norm_ep_avg.strain==strain),'iptg'],
                naive_GFP_norm_ep_avg.loc[(naive_GFP_norm_ep_avg.strain==strain),'Measurement'],
               yerr=naive_GFP_norm_ep_avg.loc[(naive_GFP_norm_ep_avg.strain==strain),'std'],
               capsize=2,color=sns.color_palette()[i],linestyle='None',marker='s',ms=5)
    
# ax.set_ylim(0,2)
ax.legend(naive_GFP_norm_ep.strain.unique(),loc='best')
ax.set_xlabel('IPTG ($\mu$M)',fontsize=14)
ax.set_ylabel('endpoint GFP/OD',fontsize=12)
ax.legend(['naive 1x','naive 2x','naive 1x avg','naive 2x avg'],fontsize=12,loc='best')
plt.savefig('20210821_naive1x2x_HCK_epGFP_20220805.pdf')